In [9]:
import heapq
import os


class Binarytreenode:
    def __init__(self,value,freq):               #creates node with key and value , with left and right child
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
        
    def __lt__(self,other):                      #overloading lessthan function for comparision
        return self.freq < other.freq
    
    def __eq__(self,other):                      #overloading equal function for comparision
        return self.freq == other.freq
        
class HuffmanCoding:
    
    def __init__(self,path):
        self.path = path               #provides path of the file 
        self.__heap = []               #empty heap list
        self.__codes = {} 
        self.__reverseCodes = {}
        
    def __make_frequency_dict(self,text):
        freq_dict = {}                 #empty frequency dictionary
        for char in text:             
            if char not in freq_dict:
                freq_dict[char] = 0
            freq_dict[char] += 1
        return freq_dict
     
    
    def __buildheap(self,freq_dict):
        
        for key in freq_dict:                                      
            frequency = freq_dict[key]
            binary_tree_node = Binarytreenode(key,frequency)
            heapq.heappush(self.__heap,binary_tree_node)
        pass
        
    def __buildTree(self):
        while len(self.__heap) > 1:           #there should be 2 or more nodes,cause we need to compare with 2 nodes
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            freq_sum = binary_tree_node_1.freq + binary_tree_node_2.freq
            newnode = Binarytreenode(None,freq_sum)
            
            newnode.left = binary_tree_node_1
            newnode.right = binary_tree_node_2
            
            heapq.heappush(self.__heap,newnode)
            
        return 
    
    
    def __buildCodeshelper(self,root,curr_bits):
        
        if root is None:
            return
        
        if root.value is not None:
            self.__codes[root.value] = curr_bits
            self.__reverseCodes[curr_bits] = root.value
            return
        
        self.__buildCodeshelper(root.left,curr_bits+"0")
        self.__buildCodeshelper(root.right,curr_bits+"1")
        
        
    def __buildCodes(self):
        root = heapq.heappop(self.__heap)
        self.__buildCodeshelper(root,"")
    
    
    def __getEncodedText(self,text):
        encoded_text = ""
        for char in text:
            encoded_text += self.__codes[char]
        return encoded_text
    
    
    def __getPaddedEncodedText(self,encoded_text):
        
        padded_amount = 8-(len(encoded_text)%8)
        
        """this formula (8-len(encoded_text)%8) helps to find the number of "0" to be added at end
           for eg: we get the encoded text as 100010101101 so we need to make pair of 8 bits to convert it
           into bytes so we have to add the "0's" to make the remaining 8-bit pair
        """
        
        for i in range(padded_amount):
            encoded_text += '0'                #this loop will append all the 0 we needed
        
#         import pdb
#         pdb.set_trace()
        padded_info = "{0:08b}".format(padded_amount)
        # {"0:08b"} this takes the 0th element from padded_amount from "0:" and covert into 8bit binary using
        #  ":08b" eg. 5 binary is 101 so it will convert into 8 bit binary ,i.e 0000 0101
        
        padded_encoded_text = padded_info + encoded_text
        return padded_encoded_text
    
    def __getBytesArray(self,padded_encoded_text):
        
        array = []
        
        for i in range(0,len(padded_encoded_text),8):
            byte = padded_encoded_text[i:i+8]
            array.append(int(byte,2))   #we need to convert the string of 0's and 1's to integer
    
        return array
    
    def compress(self):
        #get file from path
        #read text from the file
        file_name,file_extension = os.path.splitext(self.path)
        output_path = file_name + ".bin"
        
        with open(self.path,'r+') as file , open(output_path,'wb') as output:  #"r+" is read/write mode and "wb" is in binary mode
            #make frequency dictionary using the text
            text = file.read()
            text = text.rstrip()

            freq_dict = self.__make_frequency_dict(text)

            #construct the heap from the frequency dictionary
            self.__buildheap(freq_dict) 

            #construct the binary tree from the heap
            self.__buildTree()

            #construct the codes from binary tree
            self.__buildCodes()

            #creating the encoded text using the codes
            encoded_text = self.__getEncodedText(text)

            #put encoded text into binary file

            #pad this encoded text
            padded_encoded_text = self.__getPaddedEncodedText(encoded_text)

            bytes_array = self.__getBytesArray(padded_encoded_text)
            #rerturn binary file as output
            final_bytes = bytes(bytes_array)
            output.write(final_bytes)

            print("Compressed")
            return output_path
    
    
    def __removePadding(self,text):
        padded_info = text[:8]
        extra_padding = int(padded_info,2)
        
        text = text[8:]
        text_after_padding_removed = text[:-1*extra_padding]
        return text_after_padding_removed
    
    
    def __decodeText(self,text):
        decoded_text = ""
        curr_bits = ""
        
        for bit in text:
            curr_bits += bit
            if curr_bits in self.__reverseCodes:
                character = self.__reverseCodes[curr_bits]
                decoded_text += character
                curr_bits = ""
        return decoded_text
        
    
    def decompress(self,input_path):
        file_name,file_extension = os.path.splitext(self.path)
        output_path = file_name + "_decompressed" + ".txt"
        with open(input_path,'rb') as file, open(output_path,'w') as output:
            bit_string = ""
            byte = file.read(1)
            while byte:
                byte = ord(byte)
                bits =  bin(byte)[2:].rjust(8,'0')
                bit_string += bits
                byte = file.read(1)
            #remove padding
            actual_text = self.__removePadding(bit_string)
            decompressed_text = self.__decodeText(actual_text)
            output.write(decompressed_text)
            print("Decompressed")
        return
    
path = '/home/pawan/Desktop/Vivek/Data-Structure-Using-Python/Huffman_coding/test_file.txt'
h = HuffmanCoding(path)
output_path = h.compress()
h.decompress(output_path)

Compressed
Decompressed
